In [1]:
import os
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import h5py
import numpy as np

2024-03-04 09:45:43.020221: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 09:45:43.038601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 09:45:43.038617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 09:45:43.039075: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 09:45:43.042213: I tensorflow/core/platform/cpu_feature_guar

# Load the trained model

In [2]:
model_dir = os.path.join(os.getcwd(), 'EEGInception/model')
print(model_dir)
loaded_model = tf.saved_model.load(model_dir)
predict_fn = loaded_model.signatures['serving_default']


/home/brgonzalez/Escritorio/EEG_CHANEL/EEGInception/model


2024-03-04 09:45:44.512646: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-04 09:45:44.538782: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-04 09:45:44.538865: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Load the data

In [2]:
dataset_path = os.path.join(os.getcwd(), 'data/UVA-DATASET/archive/GIB-UVA ERP-BCI.hdf5')

hf = h5py.File(dataset_path, 'r')
features = np.array(hf.get("features"))
erp_labels = np.array(hf.get("erp_labels"))
codes = np.array(hf.get("codes"))
trials = np.array(hf.get("trials"))
sequences = np.array(hf.get("sequences"))
matrix_indexes = np.array(hf.get("matrix_indexes"))
run_indexes = np.array(hf.get("run_indexes"))
subjects = np.array(hf.get("subjects"))
database_ids = np.array(hf.get("database_ids"))
target = np.array(hf.get("target"))
matrix_dims = np.array(hf.get("matrix_dims"))
hf.close()

NameError: name 'os' is not defined

Split the data

In [1]:
split_index = int(len(features) * 0.8)

X_train = features[:split_index]
y_train = erp_labels[:split_index]

X_test = features[split_index:]
y_test = erp_labels[split_index:]

NameError: name 'features' is not defined

# Evaluate the model

In [5]:
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_true = []
y_pred = []

for i in range(len(X_test)):
    instance = X_test[i].reshape(1, X_test[i].shape[0], X_test[i].shape[1], 1) 
    model_input = tf.convert_to_tensor(instance, dtype=tf.float32)  
    predictions = predict_fn(model_input)

    predicted_label = np.argmax(predictions['dense'].numpy(), axis=-1)
    y_pred.append(predicted_label)
    
    true_label = np.argmax(y_test[i])
    y_true.append(true_label)

2024-03-04 09:45:50.354184: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8800


Calculate some metrics

In [6]:
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print('Accuracy:', acc)
print('F1:', f1)
print('Confusion Matrix:', cm)

Accuracy: 0.9591086279512269
F1: 0.0
Confusion Matrix: [[134585   5738]
 [     0      0]]
